In [2]:
!pip install torch 
!pip install psycopg2
!pip install rank_bm25
!pip install langchain 
!pip install langchain_community
!pip install langchain_openai
!pip install sentencepiece
!pip install transformers 
!pip install datasets
!pip install accelerate 
%env SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL True
!pip install -U sklearn
!pip install spacy
!python3 --version
!pip install -U transformers 

  Using cached langchain_core-0.2.12-py3-none-any.whl (355 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.11 requires langchain-core<0.2,>=0.1.29, but you have langchain-core 0.2.12 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.12 which is incompatible.
env: SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True
Python 3.10.8


In [2]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn 
from sklearn.model_selection import train_test_split 

import json 
import copy 
import gc 
import os 
import re 
from collections import defaultdict
from pathlib import Path 

from transformers import AutoTokenizer 


from spacy.lang.en import English 
from transformers.tokenization_utils import PreTrainedTokenizerBase 
from transformers.models.deberta_v2 import (
    DebertaV2ForTokenClassification,
    DebertaV2TokenizerFast,
)
from transformers.trainer import Trainer 
from transformers.training_args import TrainingArguments
from transformers.trainer_utils import EvalPrediction 
from transformers.data.data_collator import DataCollatorForTokenClassification
from datasets import (
    Dataset, 
    DatasetDict, 
    concatenate_datasets,
    features
)
from transformers import AutoConfig

from transformers import TextClassificationPipeline

import argparse 
from itertools import chain 
from functools import partial 

from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding

import random 

#SQL Agent imports 
#RAG
from PyPDF2 import PdfReader
from transformers import AutoTokenizer, AutoModel 
import torch 
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import sentence_transformers
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

import warnings 
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Initialize routing agent from saved model 

In [3]:
#Please provide the pre-trained model path
modelpath = '/root/checkpoint-894'
model = AutoModelForSequenceClassification.from_pretrained(modelpath)
tokenizer = AutoTokenizer.from_pretrained(modelpath)

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
def routing_agent(user_question):
    prediction = pipe(user_question)
    print(prediction)
    if prediction[0]['label'] == 'LABEL_1':
        sql_result = sql_answer(user_question)
        return sql_result
    if prediction[0]['label'] == 'LABEL_0':
        tdr_result = "This is a tdr question"
        answer = create_RetrievalQA_chain(question,model,thread_bm25_store,prompt,'bm25',30) 
        return answer


# Add SQL Agent  

In [74]:
#functions 
def get_vector_store(filename):
    filename = filename
    pdf_reader = PdfReader(filename)
    
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
        
    pdf_docs = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 5000, chunk_overlap = 200)

    for idx, page in enumerate(pdf_reader.pages):
        if len(text) > 0:
            pdf_docs.extend(
                text_splitter.create_documents(
                    texts = [text],
                    metadatas = [{'filename': filename, 'page': idx+1}]
                )
            )
            
    embedding = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return FAISS.from_documents(pdf_docs, embedding)

def get_schema(_):
    schema = db.get_table_info()
    return schema


def run_query(query):
    return db.run(query)

def sql_answer(user_question):
    # add Try to run SQL 5 times 
    error_counter = 0 
    while error_counter < 5:
        try:
            question = user_question
            ss_result = v_db.similarity_search(question)
            top_ss_docs = ss_result[0:1]
            context = " ----- ".join([ss_result.page_content for ss_result in top_ss_docs])  
            return full_chain.invoke({"context": context,"question": user_question}).content
        except:
            error_counter += 1 
    result = "Unable to generate answer based on the question being asked. please try again with a different question."
    return result 
    

In [75]:
# RAG
v_db = get_vector_store('data_dict.pdf')

In [76]:
# SQL Agent 

In [77]:
# setup database and schema
sqlite_uri = 'sqlite:///./snyth.db' 
db = SQLDatabase.from_uri(sqlite_uri)

In [78]:
#setup model 
key = 'sk-proj-WVAYZM6xMzLkvMYc3p2pT3BlbkFJnlTUh1OzdtEVL1TT7Aq8'
sql_llm = ChatOpenAI(openai_api_key=key)

In [79]:
#sql chain 
template = """Based on the table schema and context below, write a SQL query that would answer the user's question:
{schema}
{context}
Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [80]:
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | sql_llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [81]:
#full chain 
template = """Based on the table schema below, question, sql query, and sql response, and context write a natural language response:
{schema}
{context}
Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [82]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | sql_llm
)

In [84]:
# user_question = 'what is the most likly vulnerablity source?'
user_question = 'How many apples are in the dog table?'
sql_answer(user_question)

'Unable to generate answer based on the question being asked. please try again with a different question.'

# Add TDR Agent

In [15]:
import boto3
from langchain_community.chat_models import BedrockChat

from boto3 import client
from botocore.config import Config
def load_model():
    config = Config(read_timeout=1000)

    bedrock_runtime = boto3.client(service_name='bedrock-runtime', 
                          region_name='us-east-1',
                          config=config)

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    
    model_kwargs = { 
        "max_tokens": 100000,
        "temperature": 0,
        "top_k": 250,
        "top_p": 1,
        "stop_sequences": ["\n\nHuman"],
        
    }
    
 
    
    model = BedrockChat(
        client=bedrock_runtime,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )

    return model

llm = load_model()


In [16]:
import boto3
from langchain_community.chat_models import BedrockChat

from boto3 import client
from botocore.config import Config

In [17]:
from langchain import PromptTemplate


def load_prompt():
    with open(f'prompt_template/final_prompt.txt', 'r') as file:
        data = file.read()
    prompt_template = data
    prompt = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
    return prompt
prompt = load_prompt()

In [18]:
def load_model():
    config = Config(read_timeout=1000)

    bedrock_runtime = boto3.client(service_name='bedrock-runtime', 
                          region_name='us-east-1',
                          config=config)
    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    
    model_kwargs = { 
        "max_tokens": 100000,
        "temperature": 0,
        "top_k": 250,
        "top_p": 1,
        "stop_sequences": ["\n\nHuman"],
        
    }
    
 
    
    model = BedrockChat(
        client=bedrock_runtime,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )

    return model
model = load_model()

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA

def create_RetrievalQA_chain(query,model,store,prompt,store_type,no_of_doc_per_batch):
    print("Connecting to bedrock")
    if store_type == "Vector store":
        retriever = store.as_retriever(
            search_type="mmr", search_kwargs={"k": 30,"include_metadata": True}
        )
    elif store_type == "bm25":
        retriever  = store
        retriever.k = no_of_doc_per_batch
        retriever.get_relevant_documents(query)
        
    chain = RetrievalQA.from_chain_type(
        llm=model,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=False,
        chain_type_kwargs={"prompt": prompt, 
                          }
    )
    
    result = chain.invoke({"query": query,
                                           
                          })
 
    return result


In [20]:
from langchain.retrievers import BM25Retriever
def generate_thread_sql(collection_id,cve_result_filter):
    collection_ids = collection_id
    params = []
    collection_ids_tuple = tuple(collection_ids)
    params.append(collection_ids_tuple)
    
    
    prefinal_sql = f"""
       SELECT   * 
                 FROM 
                     langchain_pg_embedding lpe 
                 WHERE 
                     lpe.collection_id in (
                         SELECT 
                             lpc.uuid 
                         FROM
                             langchain_pg_collection lpc
                        WHERE
                             lpc.name in  %s
                     )
    """

    for key, value in cve_result_filter.items():
        
        if len(value) == 0:
            pass
        else:
                prefinal_sql = prefinal_sql.strip() + f"""
                             AND
                             lpe.cmetadata ->> '{key}' in %s
                            """
                params.append(tuple(value))
                
    prefinal_sql = prefinal_sql.strip() + """
    order by (lpe.cmetadata->>'_time')::timestamptz desc
    """
    
    
    sql = f"""
            SELECT document FROM (
            {prefinal_sql.strip()}
            ) as subquery  
    """
    params = tuple(params)

    return sql,params



def get_documents_for_thread(collection_id, filter_dict,conn_for_vector_db):
    
    
    sql,params = generate_thread_sql(collection_id,filter_dict,)

    with conn_for_vector_db.cursor() as cur:
        
        cur.execute(sql, params)

        rows = cur.fetchall()
        bm25_documents = [row[0] for row in rows] 
        conn_for_vector_db.commit()
        conn_for_vector_db.close()

    bm25_store = BM25Retriever.from_texts(bm25_documents)
    
    return bm25_store


In [21]:
#collection names created for the final results
collection_id = ["TDR_Analysis"]

In [22]:
onprem_cloud_filter = {
  'host': []
}

In [23]:
import boto3
import json
import logging
import psycopg2
import psycopg2.extras as psycopg2_extras
from psycopg2 import pool
from pprint import pprint
conn_for_vector_db = psycopg2.connect(
    dbname="postgres",
    user="postgres", 
    password= "password1", 
    host="database-2-instance-1.cnzoukwx6fwf.us-east-1.rds.amazonaws.com"
)

In [24]:
thread_bm25_store = get_documents_for_thread(collection_id, onprem_cloud_filter,conn_for_vector_db)

In [33]:

# question = "Analyze security threat analysis reports for the hosts and provide a comprehensive report of the threat that takes into account individual host based reports."
# question = 'what is the most likly vulnerablity source and who owns the most vulnerablity from that source?'
question = 'How many apples are there in the dogs tables'

In [36]:
output = routing_agent(question)
output

[{'label': 'LABEL_1', 'score': 0.9999982118606567}]


In [ ]:
class ConversationMemory:
    def __init__(self):
        self.memory = {}

    def get_memory(self, session_id):
        return self.memory.get(session_id, [])

    def update_memory(self, session_id, message):
        if session_id not in self.memory:
            self.memory[session_id] = []
        self.memory[session_id].append(message)


In [ ]:
class SQLAgent:
    def query(self, user_input, history):
        # Include the SQL querying logic, using history as needed
        return 'SQL Agent Response based on user input and history'


In [ ]:
class TDRAgent:
    def analyze(self, user_input, history):
        # Include the threat analysis logic, using history as needed
        return 'TDR Agent Response based on user input and history'


In [ ]:
class RoutingAgent:
    def __init__(self, sql_agent, tdr_agent, memory):
        self.sql_agent = sql_agent
        self.tdr_agent = tdr_agent
        self.memory = memory

    def route(self, session_id, user_input):
        # Retrieve the conversation history for this session
        history = self.memory.get_memory(session_id)

        # Append the current user input to the history
        history.append(user_input)

        # Decide which agent to use based on the user input and context
        if 'SQL' in user_input:
            response = self.sql_agent.query(user_input, history)
        else:
            response = self.tdr_agent.analyze(user_input, history)

        # Update the memory with the agent's response
        self.memory.update_memory(session_id, response)

        return response


In [ ]:
# Initialize the agents and memory
memory = ConversationMemory()
sql_agent = SQLAgent()
tdr_agent = TDRAgent()
routing_agent = RoutingAgent(sql_agent, tdr_agent, memory)

# Simulate a conversation
session_id = 'user_123'
print(routing_agent.route(session_id, 'Run an SQL query'))
print(routing_agent.route(session_id, 'Analyze this log file'))
